## Simple Mission

This notebook serves to check the ardupilot installation

In [1]:
import os
import subprocess
import time
from typing import cast

import pymavlink.dialects.v20.ardupilotmega as mavlink
from pymavlink import mavutil

from config import ARDUPILOT_VEHICLE_PATH, LOGS_PATH, VEH_PARAMS_PATH
from helpers import clean
from mavlink.customtypes.mission import MissionLoader
from mavlink.enums import CmdNav, MissionResult
from mavlink.util import connection
from plan import State
from plan.planner import Plan

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agrre with first waypoint in mission.waypoint
spawn_str= '47.397742,8.545594,0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I0 --sysid 1 "
    f"--no-rebuild "
    f"--use-dir={LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f'--custom-location={spawn_str}'
)


# Open a new terminal and run the command
subprocess.Popen(["gnome-terminal","--title=planner","--", 
                  "bash", "-c", f"{vehicle_cmd}; exec bash"])

<Popen: returncode: None args: ['gnome-terminal', '--title=planner', '--', '...>

## 1. Connect to the vehicle

In [3]:
time.sleep(1)
conn = connection("tcp:127.0.0.1:5760")
conn.wait_heartbeat() 
print("✅ Heartbeat received")


✅ Heartbeat received


# 2. Load the waypoint file

In [4]:
mission = MissionLoader(conn.target_system, conn.target_component)
count = mission.load("plan/missions/simple_mission.waypoints") # your QGC WPL 110 file
print(f"✅ Loaded {count} waypoints")

✅ Loaded 4 waypoints


# 3. Read the mission

In [5]:
# Retrieve each mission item
for i in range(count):
    wp = mission.item(i)
    cmd_name = CmdNav(wp.command).name
    print(
        f"🧭 Mission[{i}] → cmd: {cmd_name}, "
        f"x: {wp.x}, y: {wp.y}, z: {wp.z}, current: {wp.current}"
    )

🧭 Mission[0] → cmd: WAYPOINT, x: 47.397742, y: 8.545594, z: 0.0, current: 0
🧭 Mission[1] → cmd: TAKEOFF, x: 47.397742, y: 8.545594, z: 10.0, current: 0
🧭 Mission[2] → cmd: WAYPOINT, x: 47.397787, y: 8.545594, z: 10.0, current: 0
🧭 Mission[3] → cmd: LAND, x: 47.397787, y: 8.545594, z: 0.0, current: 0


# 3. Clear any existing mission

In [6]:
conn.mav.mission_clear_all_send(conn.target_system, conn.target_component)
ack = conn.recv_match(type='MISSION_ACK', blocking=True, timeout=5)
print(f"🧹 Cleared previous mission: {ack.type if ack else 'no ack'}")
time.sleep(1)

🧹 Cleared previous mission: 0


# 4. Send new mission count

In [7]:
conn.mav.mission_count_send(conn.target_system, conn.target_component, mission.count())
print(f"📤 Sending {mission.count()} mission items")


📤 Sending 4 mission items


# 5. Send each item in response to MISSION_REQUEST

In [8]:
for i in range(mission.count()):
    msg = conn.recv_match(type='MISSION_REQUEST', blocking=True, timeout=5)
    if not msg or msg.seq != i:
        raise RuntimeError(f"❌ Unexpected mission request: {msg}")
    conn.mav.send(mission.wp(i))
    print(f"✅ Sent mission item {i}")

✅ Sent mission item 0
✅ Sent mission item 1
✅ Sent mission item 2
✅ Sent mission item 3


# 6. Wait for final MISSION_ACK

In [9]:
ack = conn.recv_match(type='MISSION_ACK', blocking=True, timeout=5)
if ack and MissionResult(ack.type) == MissionResult.ACCEPTED:
    print("🎉 Mission upload successful!")
else:
    print(f"⚠️ Mission upload failed or timed out: {ack}")

🎉 Mission upload successful!


## Check the mission

In [10]:
# Create a mission protocol loader
wp = MissionLoader()


# Request list of mission items
conn.mav.mission_request_list_send(conn.target_system, conn.target_component)

# Wait for mission count
msg = conn.recv_match(type='MISSION_COUNT', blocking=True, timeout=5)
if msg:
    mission_count = msg.count
    print(f"📦 UAV has {mission_count} mission items")
    # Retrieve each mission item
    for i in range(mission_count):
        conn.mav.mission_request_send(conn.target_system, conn.target_component, i)
        item = conn.recv_match(type='MISSION_ITEM', blocking=True, timeout=5)
        if item:
            wp.set(item, i)
            print(
                f"🧭 Mission[{i}] → cmd: {CmdNav(item.command).name}, "
                f"x: {item.x}, y: {item.y}, z: {item.z}, current: {item.current}"
            )
else:
    print('No mission count received')




📦 UAV has 4 mission items
🧭 Mission[0] → cmd: WAYPOINT, x: 0.0, y: 0.0, z: 0.0, current: 0
🧭 Mission[1] → cmd: TAKEOFF, x: 47.397743225097656, y: 8.545594215393066, z: 10.0, current: 0
🧭 Mission[2] → cmd: WAYPOINT, x: 47.39778518676758, y: 8.545594215393066, z: 10.0, current: 0
🧭 Mission[3] → cmd: LAND, x: 47.39778518676758, y: 8.545594215393066, z: 0.0, current: 0


# Execute Plan

In [12]:
plan_auto = Plan.auto(mission_name="simple_mission")
plan_auto.bind(conn)
while plan_auto.state != State.DONE:
    plan_auto.act()

Vehicle 1: ▶️ Plan Started: 📋 
Vehicle 1: ▶️ Action Started: 📤🚁  UPLOAD_MISSION
Vehicle 1: ▶️ Step Started: clear uav missions
Vehicle 1: ✅ Step Done: clear uav missions
✅ 4 waypoints read.
✅ Sent mission item 0
✅ Sent mission item 1
✅ Sent mission item 2
✅ Sent mission item 3
Vehicle 1: ▶️ Step Started: load mission simple_mission
🎉 Mission upload successful!
Vehicle 1: ✅ Step Done: load mission simple_mission
Vehicle 1: ✅ Action Done: 📤🚁  UPLOAD_MISSION
Vehicle 1: ▶️ Action Started: 🔧 PREARM
Vehicle 1: ▶️ Step Started: Check disarmed
Vehicle 1: ✅ Step Done: Check disarmed
📡 Requested message EKF_STATUS_REPORT at 1.00 Hz
Vehicle 1: ▶️ Step Started: Check EKF status
Vehicle 1: ✅ Step Done: Check EKF status
📡 Requested message GPS_RAW_INT at 1.00 Hz
Vehicle 1: ▶️ Step Started: Check GPS
Vehicle 1: ✅ Step Done: Check GPS
📡 Requested message SYS_STATUS at 1.00 Hz
Vehicle 1: ▶️ Step Started: Check system
Vehicle 1: ✅ Step Done: Check system
Vehicle 1: ✅ Action Done: 🔧 PREARM
Vehicle 1: ▶️ 

# Monitoring

In [13]:

conn.mav.request_data_stream_send(
    conn.target_system,
    conn.target_component,
    mavutil.mavlink.MAV_DATA_STREAM_POSITION,  # Stream ID
    4,     # Frequency in Hz
    1      # Start (1 to enable streaming)
)


last_seq = mission.count() - 1

while True:
    msg = conn.recv_match(blocking=True, timeout=1)

    if not msg:
        print("⏳ Waiting for mission update...")
        continue

    # ✅ Reached a waypoint
    if msg.get_type() == "MISSION_ITEM_REACHED":
        msg = cast(mavlink.MAVLink_mission_item_reached_message, msg)
        print(f"📌 Reached waypoint: {msg.seq}")
        if msg.seq == last_seq:
            print("✅ Final waypoint reached")

    # ✅ UAV position
    elif msg.get_type() == "GLOBAL_POSITION_INT":
        msg = cast(mavlink.MAVLink_global_position_int_message, msg)
        lat = msg.lat / 1e7
        lon = msg.lon / 1e7
        alt = msg.relative_alt / 1000.0
        print(f"📍 Position: lat={lat:.7f}, lon={lon:.7f}, alt={alt:.2f} m")

    # ✅ Look for end hints in text
    elif msg.get_type() == "STATUSTEXT":
        msg = cast(mavlink.MAVLink_statustext_message, msg)
        text = msg.text.strip().lower()
        if "disarming" in text:
            print("🏁 Mission completed")
            break

📍 Position: lat=47.3977420, lon=8.5455940, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.00 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.02 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.08 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.18 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.46 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=0.74 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=1.07 m
📍 Position: lat=47.3977420, lon=8.5455939, alt=1.46 m
📍 Position: lat=47.3977420, lon=8.5455938, alt=1.92 m
📍 Position: lat=47.3977420, lon=8.5455938, alt=2.44 m
📍 Position: lat=47.3977420, 